<a href="https://colab.research.google.com/github/ShakutaiGit/Final_Project_Applied_Data_science/blob/main/Souce_Code_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

relevant imports 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental import preprocessing
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model 
from tensorflow.python.keras.utils import np_utils
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
import numpy as np
import pandas as pd 
!pip install optuna
import optuna
from sklearn.model_selection import KFold
from skimage.transform import resize
from skimage.transform import rescale
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
import cv2
from statistics import mean
from google.colab import files
import time 
import csv


     |████████████████████████████████| 307kB 7.7MB/s 
     |████████████████████████████████| 174kB 13.1MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 112kB 15.5MB/s 
     |████████████████████████████████| 143kB 14.0MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=c28e7f1af70bc107841e1b4a38007873661f4d9ad65e06d7baca5ff0136693b2
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


global variables

In [2]:
optimizer = None
size = 96 
data_shape = None
batch_size = 128 # like the original paper 
classes = 0
AUTOTUNE = tf.data.AUTOTUNE
x_train = []
x_test = []
y_train = []
y_test = []
algorithem_name = 'source'
database_name = None

loading the dataset and pre processing 

In [3]:
train_ds ,train_info= tfds.load('oxford_flowers102', split='train[:80%]', with_info=True)
test_ds , test_info = tfds.load('oxford_flowers102', split='train[80%:]', with_info=True)
## pre processing 
database_name = train_info.name
resize_and_rescale = tf.keras.Sequential([
layers.experimental.preprocessing.Resizing(size, size),
layers.experimental.preprocessing.Rescaling(1./255)
])

def pre_process(data):
  pictures=[]
  labels = []
  for i in data:
    temp = resize_and_rescale(i['image'])
    temp = np.asarray(temp)
    pictures.append(temp)
    labels.append(int(i['label']))
  return np.asarray(pictures),np.asarray(labels)

x_train,y_train =pre_process(train_ds)
data_shape= x_train[0].shape

x_test,y_test =pre_process(test_ds)
classes = train_info.features['label'].num_classes



Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteK3Z78Y/oxford_flowers102-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteK3Z78Y/oxford_flowers102-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/oxford_flowers102/2.1.1.incompleteK3Z78Y/oxford_flowers102-validation.tfrecord


Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
(96, 96, 3)


creating the model as described in the report

In [4]:
def creating_the_model():
  # creating input layer
  inputs = layers.Input(shape=(size, size, 3))
  # loading the pretrained model 
  pre_trained_model = keras.applications.ResNet50V2(include_top=False, weights='imagenet',input_tensor=inputs,pooling='avg')
  pre_trained_model.trainable= False
  #normlizing the out put of the model 
  x = layers.BatchNormalization()(pre_trained_model.output)
  top_dropout_rate = 0.5
  # adding dropout layer
  x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
  outputs = layers.Dense(classes, activation="softmax", name="pred")(x)
  model = tf.keras.Model(inputs, outputs, name="ResNet")
  #unfreeze some layers of the pretrained model
  for layer in model.layers[-10:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
  return model



the obejctive function that describe in the report

In [5]:
def objective(trial,outer_x_train,outer_y_train):
 # inner cross validation loop
  internal_kfold = KFold(n_splits=3, shuffle=True)
  fold_acc = []  
  for inner_train_index,inner_test_index in internal_kfold.split(outer_x_train,outer_y_train):
    inner_x_train, inner_x_test = outer_x_train[inner_train_index], outer_x_train[inner_test_index]
    inner_y_train, inner_y_test = outer_y_train[inner_train_index], outer_y_train[inner_test_index]
    model = creating_the_model()
    # hyper paramters 
    model.compile(
    loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(
        learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(
  initial_learning_rate=trial.suggest_float(name="lr", low = 0.00001,high= 0.1, log=True),
  first_decay_steps=trial.suggest_float(name="steps", low = 1,high=1000), 
  t_mul=trial.suggest_categorical(name="t_mult", choices =[1,2]), 
  m_mul=trial.suggest_float(name="m_mult", low=0,high=1), 
  alpha=0.0)
      ), metrics=["accuracy"]
  )
    # training the model 
    model.fit(
        x=inner_x_train,
        y=to_categorical(inner_y_train,num_classes=classes),
        # validation_data= (inner_x_test,to_categorical(,inner_y_test)),
        # shuffle=True,
        validation_split=0.1,
        batch_size=batch_size,
        epochs=5
    )
    score = model.evaluate(inner_x_test,to_categorical(inner_y_test, num_classes=classes) , verbose=0)
    fold_acc.append(score[1] * 100)
  return mean(fold_acc)

creating dict with the metrics information so we can write easly to csv 

In [6]:
def fold_dict_creator(Hyper_Parameters_Values,score,training_time,inference_time,fold):
  fold_dict = {}
  fold_dict['Dataset Name'] = database_name
  fold_dict['Algorithm Name'] = algorithem_name
  fold_dict['Cross Validation [1-10]'] = fold
  fold_dict['Hyper-Parameters Values'] = Hyper_Parameters_Values
  fold_dict['Accuracy'] = score[1]
  fold_dict['TPR'] = score[2]
  fold_dict['FPR'] = score[3]
  fold_dict['Precision'] = score[4]
  fold_dict['AUC'] = score[5]
  fold_dict['PR-Curve'] = score[6]
  fold_dict['Training Time'] = training_time
  fold_dict['Inference Time'] = inference_time
  return fold_dict


the outer loop and the main script of the model.

In [ ]:
outer_kfold = KFold(n_splits=10, shuffle=True)
fold_num = 0
dict_data= []
# outer cross validation loop 
for outer_train_index,outer_test_index in outer_kfold.split(x_train, y_train):
  fold_num += 1
  print(fold_num)
  outer_x_train, outer_x_test = x_train[outer_train_index], x_train[outer_test_index]
  outer_y_train, outer_y_test = y_train[outer_train_index], y_train[outer_test_index]
  study = optuna.create_study(direction="maximize")
  # creating the study as described in the report 
  study.optimize(lambda trail:objective(trail,outer_x_train,outer_y_train), n_trials=50)
  model = creating_the_model()
  # compiling the model with the parameters that produce the best results 
  model.compile(
    loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(
        learning_rate=tf.keras.optimizers.schedules.CosineDecayRestarts(
  initial_learning_rate=study.best_params["lr"],
  first_decay_steps=study.best_params["steps"], 
  t_mul=study.best_params["t_mult"], 
  m_mul=study.best_params["m_mult"], 
  alpha=0.0)
        # describing which metrics we gunna use
      ), metrics=["accuracy", tf.keras.metrics.SensitivityAtSpecificity(0.5),
      tf.keras.metrics.SpecificityAtSensitivity(0.5), 
      tf.keras.metrics.Precision(),
      tf.keras.metrics.AUC(),
      tf.keras.metrics.AUC(curve = 'pr')]
  )
  start_time = time.time()
  model_info = model.fit(
    outer_x_train,
    to_categorical(outer_y_train,num_classes=classes),
    validation_split= 0.15,
    shuffle=True,
    batch_size=batch_size,
    epochs=5,
  )
  training_time = time.time() - start_time
  start_time = time.time()
  score = model.evaluate(outer_x_test,to_categorical(outer_y_test,num_classes=classes))
  # print(model.metrics_names)
  inference_time = time.time() - start_time
  dict_data.append(fold_dict_creator(str(study.best_params), score, training_time , inference_time, fold_num))
csv_file = "results_algo_{}_data_set{}.csv".format(algorithem_name,database_name)
#writing to csv 
with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=dict_data[0].keys())
    writer.writeheader()
    for data in dict_data:
        writer.writerow(data)
files.download(csv_file)  